In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip -q install accelerate transformers bitsandbytes peft langchain-huggingface langchain langchain-community crewai langchainhub langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.3/209.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
import torch
from langchain_huggingface import HuggingFacePipeline

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "sun000090/Llama_3_8B",
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(
    "sun000090/Llama_3_8B",
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,
                max_new_tokens=256,
                do_sample=True,
                return_full_text=False,
                temperature=0.5,
                top_p=0.5,
                top_k=20)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [ ]:
template = """System: You're a helpful converastion chatbot. Keep your response to limit of 100 words only.
Question: {question}
Answer:"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]))
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "How is overall weather?"

In [ ]:
chain.invoke({'question':question})

" The overall weather is mostly sunny with a few clouds. It's a beautiful day with a high of 75 degrees Fahrenheit. Perfect weather for a walk or outdoor activities. Enjoy your day!"

In [ ]:
from crewai import Agent, Task, Crew, Process
import os

In [ ]:
os.environ["OPENAI_API_BASE"] = 'https://api.groq.com/openai/v1'
os.environ["OPENAI_MODEL_NAME"] ='llama3-8b-8192'
os.environ["OPENAI_API_KEY"] ='key'

In [ ]:
# Create Agent
email_classifier = Agent(
      role='Senior Analyst I',
      goal="""You are a senior analyst I and your goal is to classify the emails as important mail, newsletter and spam""",
      backstory="""You are a senior analyst I and you have expertise on studying the content of email and upon
      anlysis you are able to classify the email into three categories which is important mail, newsletter and spam""",
      verbose=True,
      allow_delegation=False,
      # llm=llm)
)

email_responder = Agent(
      role='Senior Analyst II',
      goal="""You are a senior analyst II and your goal is reply the important mails only""",
      backstory="""You are a senior analyst II and you have expertise on framing words which are meaningful and contextual""",
      verbose=True,
      allow_delegation=False,
      # llm=llm)
)

In [ ]:
emails = "Calculate 2 to the power 2"

In [ ]:
tasks_classifier = Task(description = f""""Classify the following email: {emails}""",
                         agent = email_classifier,
                         expected_output= "Classify into one of these three categories: 1. Important mail, 2. Newsletter, 3. Spam")

tasks_responder = Task(description = f""""Respond to mail only if its important mail: {emails}""",
                         agent = email_responder,
                         expected_output= "Respond in 100 words at amx with meaningful and contextual wordings")

In [ ]:
crew = Crew(
  agents=[email_classifier,email_responder],
  tasks=[tasks_classifier,tasks_responder],
  process = Process.sequential,
  full_output=True,
  verbose=True
)

In [ ]:
result = crew.kickoff()

 [2024-07-23 23:37:50][DEBUG]: == Working Agent: Senior Analyst I
 [2024-07-23 23:37:50][INFO]: == Starting Task: "Classify the following email: Calculate 2 to the power 2


> Entering new CrewAgentExecutor chain...
Thought: I'm confident that I can analyze this email and classify it into one of the three categories: Important mail, Newsletter, or Spam. The email appears to be a simple mathematical calculation, which doesn't seem to contain any personal or sensitive information. I'll use my expertise to examine the content and determine the category that best fits this email.

Final Answer:

Important Mail

> Finished chain.
 [2024-07-23 23:37:50][DEBUG]: == [Senior Analyst I] Task output: Important Mail


 [2024-07-23 23:37:50][DEBUG]: == Working Agent: Senior Analyst II
 [2024-07-23 23:37:50][INFO]: == Starting Task: "Respond to mail only if its important mail: Calculate 2 to the power 2


> Entering new CrewAgentExecutor chain...
Thought: I now understand that I need to respond to a

In [ ]:
texts = str(result.raw)

In [ ]:
print(texts)

The calculation 2 to the power 2, also known as 2 squared or 2², yields a result of 4. This can be easily verified by multiplying 2 by itself: 2 × 2 = 4.


In [ ]:
from langchain import hub
from langchain import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain_groq import ChatGroq

In [ ]:
template = """System: You are a senior analyst and your goal is reply the
important mails only. You are an expertise on framing words which are meaningful and contextual.
The final answer should be in the following format always if mail is important.
Format reply: Hi [Sender Name], Your reply. Thanks [Your Name] [Your Designation]

Answer the following questions as best you can. You have access to the following tools:
{tools}

Use the following format:
Question: The input question you must answer
Thought: You should always think about what to do
Action: The action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: The final answer to the original input question should be as per format of system


Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [ ]:
prompt = PromptTemplate.from_template(template)

In [ ]:
print(prompt.template)

System: You are a senior analyst and your goal is reply the
important mails only. You are an expertise on framing words which are meaningful and contextual.
The final answer should be in the following format always if mail is important.
Format reply: Hi [Sender Name], Your reply. Thanks [Your Name] [Your Designation]

Answer the following questions as best you can. You have access to the following tools:
{tools}

Use the following format:
Question: The input question you must answer
Thought: You should always think about what to do
Action: The action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: The final answer to the original input question should be as per format of system


Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
llm = ChatGroq(model_name="llama3-8b-8192", groq_api_key='key', temperature=0)

In [ ]:
llm.invoke("Hi").content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [ ]:
def reply_agent(name : str) -> str:
    contents = llm.invoke(emails).content
    return contents

In [ ]:
auto_reply = Tool(
    name="Auto Reply",
    func=reply_agent,
    description="Useful for auto replying"
)

In [ ]:
agent = create_react_agent(llm=llm, tools=[auto_reply], prompt=prompt)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[auto_reply],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1,
    return_only_outputs=True,
    return_intermediate_steps=True
)

In [ ]:
emails = '''Hey Sudneep,

I hope this email finds you well.

You have been shortlisted by Uktob.ai for the next round. This will be the final round for you :)'''

In [ ]:
response_final = agent_executor.invoke({"input": emails})



> Entering new AgentExecutor chain...
Let's break it down!

Thought: This is an important email, and I need to respond to it.

Action: Auto Reply
Action Input: "Hi [Sender Name], Thank you for informing me about the shortlisting for the next round. I'm excited to move forward. Thanks, Sudneep, Senior Analyst"
Thank you so much for the email! I'm thrilled to hear that I've been shortlisted for the next round. I'm looking forward to the final round and the opportunity to take my application to the next level. Can you please provide me with more details about the next round, such as the date, time, and any necessary preparation or materials I should bring?

> Finished chain.


In [ ]:
response_final['intermediate_steps'][0][-1]

"Thank you so much for the email! I'm thrilled to hear that I've been shortlisted for the next round. I'm looking forward to the final round and the opportunity to take my application to the next level. Can you please provide me with more details about the next round, such as the date, time, and any necessary preparation or materials I should bring?"